In [1]:
from pprint import pprint
import json
import os

import qcodes as qc

qc.set_mp_method('spawn')  # force Windows behavior on mac

qc.show_subprocess_widget()
from toymodel import AModel, MockGates, MockSource, MockMeter, AverageGetter, AverageAndRaw

model = AModel()
gates = MockGates('gates', model=model)
source = MockSource('source', model=model)
meter = MockMeter('meter', model=model)

station = qc.Station(gates, source, meter, meter.amplitude)
station.set_measurement(meter.amplitude)
c0, c1, c2, vsd = gates.chan0, gates.chan1, gates.chan2, source.amplitude


<IPython.core.display.Javascript object>

In [2]:
# Parameter snapshot
pprint(source.amplitude.snapshot())

{'__class__': 'qcodes.instrument.parameter.StandardParameter',
 'instrument': 'MockSource',
 'label': 'Source Amplitude (μV)',
 'name': 'amplitude',
 'ts': '2016-04-29 01:49:14',
 'units': '',
 'value': 0.1}


In [3]:
# Instrument snapshot
pprint(source.snapshot())

{'IDN': {'firmware': None, 'model': None, 'serial': None, 'vendor': None},
 '__class__': 'toymodel.MockSource',
 'functions': {},
 'name': 'source',
 'parameters': {'amplitude': {'__class__': 'qcodes.instrument.parameter.StandardParameter',
                              'instrument': 'MockSource',
                              'label': 'Source Amplitude (μV)',
                              'name': 'amplitude',
                              'ts': '2016-04-29 01:49:14',
                              'units': '',
                              'value': 0.1}}}


In [4]:
# Sweep_values snapshot
vals = c0[0,3,0.2]
pprint(vals.snapshot())

[{'len': 3, 'max': 3, 'min': 0, 'type': 'sequence'}]


In [5]:
# Sweep_values snapshot
vals = c0.sweep(0,3,0.2)
pprint(vals.snapshot())

[{'num': None, 'start': 0, 'step': 0.2, 'stop': 3, 'type': 'sweep'}]


In [6]:
# This is an qc.loop, here the snapshot is broken, it does not contain the data snapshot.
lp = qc.Loop(c0.sweep(0,1,0.2),0)
lp.run()
pprint(lp.snapshot())

DataSet: DataMode.PULL_FROM_SERVER, location='2016-04-29/01-49-14'
   chan0: chan0
   amplitude: amplitude
started at 2016-04-29 01:49:15
{'loop': {'delay': 0,
          'values': [{'num': None,
                      'start': 0,
                      'step': 0.2,
                      'stop': 1,
                      'type': 'sweep'}]}}


In [7]:
# Here the qc.loop becomes an active loop, now the snapshot includes the actions (wait)
# and data: None
lp = qc.Loop(c0.sweep(0,1,0.2),0).each(qc.Wait(1))
pprint(lp.snapshot())

{'data': None,
 'loop': {'actions': [{'delay': 1, 'type': 'Wait'}],
          'delay': 0,
          'sweep_values': [{'num': None,
                            'start': 0,
                            'step': 0.2,
                            'stop': 1,
                            'type': 'sweep'}]}}


In [9]:
# When we run the loop, the data snap is updated
lp.run()
pprint(lp.snapshot())

DataSet: DataMode.PULL_FROM_SERVER, location='2016-04-29/01-49-34'
   chan0: chan0
started at 2016-04-29 01:49:34
{'data': {'__class__': 'qcodes.data.data_set.DataSet',
          'arrays': [{'array.name': 'chan0', 'array_id': 'chan0'}],
          'base_location': '/Users/Merlin/Google '
                           'Drive/git/Qcodes/docs/examples',
          'formatter': 'qcodes.data.format.GNUPlotFormat',
          'io': '<DiskIO, base_location=/Users/Merlin/Google '
                'Drive/git/Qcodes/docs/examples>',
          'location': '2016-04-29/01-49-34',
          'mode': '<DataMode.PUSH_TO_SERVER: 2>'},
 'loop': {'actions': [{'delay': 1, 'type': 'Wait'}],
          'delay': 0,
          'sweep_values': [{'num': None,
                            'start': 0,
                            'step': 0.2,
                            'stop': 1,
                            'type': 'sweep'}]}}


In [10]:
# When we add more actions:
lp = qc.Loop(c0.sweep(0,1,0.2),0).each(c0, c1, c2)
pprint(lp.snapshot())

{'data': None,
 'loop': {'actions': [{'__class__': 'qcodes.instrument.parameter.StandardParameter',
                       'instrument': 'MockGates',
                       'label': 'Gate Channel 0 (mV)',
                       'name': 'chan0',
                       'ts': '2016-04-29 01:49:40',
                       'units': '',
                       'value': 1.0},
                      {'__class__': 'qcodes.instrument.parameter.StandardParameter',
                       'instrument': 'MockGates',
                       'label': 'Gate Channel 1 (mV)',
                       'name': 'chan1',
                       'ts': '2016-04-29 01:49:14',
                       'units': '',
                       'value': 0.0},
                      {'__class__': 'qcodes.instrument.parameter.StandardParameter',
                       'instrument': 'MockGates',
                       'label': 'Gate Channel 2 (mV)',
                       'name': 'chan2',
                       'ts': '2016-04-29 01

In [11]:
# When we add a second loop it is also just an action:
lp = qc.Loop(c0.sweep(0,1,0.2),0).loop(c1.sweep(0,1,0.2),0).each(c0, c1, c2)
pprint(lp.snapshot())

{'data': None,
 'loop': {'actions': [{'data': None,
                       'loop': {'actions': [{'__class__': 'qcodes.instrument.parameter.StandardParameter',
                                             'instrument': 'MockGates',
                                             'label': 'Gate Channel 0 (mV)',
                                             'name': 'chan0',
                                             'ts': '2016-04-29 01:49:40',
                                             'units': '',
                                             'value': 1.0},
                                            {'__class__': 'qcodes.instrument.parameter.StandardParameter',
                                             'instrument': 'MockGates',
                                             'label': 'Gate Channel 1 (mV)',
                                             'name': 'chan1',
                                             'ts': '2016-04-29 01:49:14',
                                             '

In [16]:
snap = lp.snapshot()

file = 'snap.json'

with open(file, 'w') as meta_file:
    json.dump(snap, meta_file, sort_keys=True, indent=4, ensure_ascii=False)